# Clone repo

In [2]:
import git
import os

def clone_github_repo(repo_url, clone_path):
    try:
        if not os.path.exists(clone_path):
            os.makedirs(clone_path)
        git.Repo.clone_from(repo_url, clone_path)
        print(f"Repository cloned to {clone_path}")
    except Exception as e:
        print(f"Failed to clone repository: {e}")



In [4]:
# Example usage
repo_url = "https://github.com/mfouesneau/NUTS"  # Replace with your repository URL

clone_path = "./repo/NUTS/"  # Path where the repository should be cloned
clone_github_repo(repo_url, clone_path)


Repository cloned to ./repo/NUTS/


# Create dict

In [7]:
def create_file_content_dict(clone_path):
    file_content_dict = {}
    for root, _, files in os.walk(clone_path):
        # Skip the .git directory
        if '.git' in root:
            continue
        for file in files:
            file_path = os.path.join(root, file)
            relative_path = os.path.relpath(file_path, clone_path)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    file_content_dict[relative_path] = f.read()
            except Exception as e:
                print(f"Failed to read {file_path}: {e}")
    return file_content_dict

In [8]:
file_content_dict = create_file_content_dict(clone_path)

{'.gitignore': '*.py[oc]\n*.sw[po]\n',
 'LICENSE': 'The MIT License (MIT)\n\nCopyright (c) 2012 Morgan Fouesneau\n\nPermission is hereby granted, free of charge, to any person obtaining a copy\nof this software and associated documentation files (the "Software"), to deal\nin the Software without restriction, including without limitation the rights\nto use, copy, modify, merge, publish, distribute, sublicense, and/or sell\ncopies of the Software, and to permit persons to whom the Software is\nfurnished to do so, subject to the following conditions:\n\nThe above copyright notice and this permission notice shall be included in all\ncopies or substantial portions of the Software.\n\nTHE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR\nIMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,\nFITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE\nAUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER\nLIABILITY, WH

# Search

In [12]:
import re

In [17]:
import re
import concurrent.futures

def search_and_format(file_content_dict, search_terms):
    results = []
    search_pattern = re.compile('|'.join(map(re.escape, search_terms)), re.IGNORECASE)

    def search_in_file(path, content):
        if search_pattern.search(path) or search_pattern.search(content):
            result = f"===\nFilename: {path}\n\nContent:\n```\n{content}\n```\n===\n"
            return result
        return None

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(search_in_file, path, content) for path, content in file_content_dict.items()]
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            if result:
                results.append(result)

    return ''.join(results)


In [ ]:
search_terms = ["sampling"]  # Add the words you want to search for
search_results = search_and_format(file_content_dict, search_terms)

print(search_results)

# Gemini API

In [1]:

import google.generativeai as genai
from dotenv import load_dotenv
import os

GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-1.5-flash')

In [21]:
repo_name = 'nov/nuts'
user_query = 'How is sampling done?'
prompt = f"Above is the relevant part of code base of a github repo {repo_name}. {user_query}"

# Normal input

In [23]:
input_to_LLM = "'''\n" + search_results + "\n'''\n" + prompt
response = model.generate_content(input_to_LLM, stream=True)
for chunk in response:
  print(chunk.text)

The
 code you provided implements the No-U-Turn Sampler (NUTS) algorithm
 for Bayesian inference. Here's a breakdown of how the sampling is done:


**1.  Hamiltonian Monte Carlo (HMC) Foundation:**

* **Energy and Dynamics:** NUTS is built upon HMC, which uses
 concepts from classical mechanics. The target distribution (the probability distribution you want to sample from) is represented as an "energy" function.
* **Gradient Information
:** HMC leverages gradient information of the energy function to guide the sampling process. It simulates the motion of a particle in this energy landscape, using Hamiltonian dynamics.

**2.  NUTS Algorithm Steps:**

* **Initial Conditions
:** The sampling starts with an initial guess of the parameters (theta0) and an initial step size (epsilon).
* **Dual Averaging:** The step size is adaptively adjusted during a burn-in period (Madapt) using
 dual averaging, which aims to find an optimal step size that balances exploration and acceptance rate.
* **Leap

# chat

In [2]:
chat = model.start_chat(history=[])

In [3]:
response = chat.send_message('hi. who are u', stream=True)
for chunk in response:
  print(chunk.text)

I
 am a large language model, trained by Google. 

Here's a
 simplified way to think about it:

* **I'm a computer program
** that's been trained on a massive amount of text data.
* **I can understand and generate human-like text.**  I can answer your
 questions, write stories, translate languages, and more. 
* **I'm not a person.** I don't have my own thoughts, feelings
, or experiences. I'm a tool that can help you with information and creativity. 

What can I help you with today? 



In [4]:
chat.history

[parts {
   text: "hi. who are u"
 }
 role: "user",
 parts {
   text: "I am a large language model, trained by Google. \n\nHere\'s a simplified way to think about it:\n\n* **I\'m a computer program** that\'s been trained on a massive amount of text data.\n* **I can understand and generate human-like text.**  I can answer your questions, write stories, translate languages, and more. \n* **I\'m not a person.** I don\'t have my own thoughts, feelings, or experiences. I\'m a tool that can help you with information and creativity. \n\nWhat can I help you with today? \n"
 }
 role: "model"]

In [5]:
chat = model.start_chat(history=chat.history)

In [6]:
chat.history

[parts {
   text: "hi. who are u"
 }
 role: "user",
 parts {
   text: "I am a large language model, trained by Google. \n\nHere\'s a simplified way to think about it:\n\n* **I\'m a computer program** that\'s been trained on a massive amount of text data.\n* **I can understand and generate human-like text.**  I can answer your questions, write stories, translate languages, and more. \n* **I\'m not a person.** I don\'t have my own thoughts, feelings, or experiences. I\'m a tool that can help you with information and creativity. \n\nWhat can I help you with today? \n"
 }
 role: "model"]

In [7]:
response = chat.send_message('i m raghav', stream=True)
for chunk in response:
  print(chunk.text)

Nice
 to meet you, Raghav! What can I do for you today
?  Do you have any questions, need help with something, or want to
 chat? 



In [8]:
chat.history

[parts {
   text: "hi. who are u"
 }
 role: "user",
 parts {
   text: "I am a large language model, trained by Google. \n\nHere\'s a simplified way to think about it:\n\n* **I\'m a computer program** that\'s been trained on a massive amount of text data.\n* **I can understand and generate human-like text.**  I can answer your questions, write stories, translate languages, and more. \n* **I\'m not a person.** I don\'t have my own thoughts, feelings, or experiences. I\'m a tool that can help you with information and creativity. \n\nWhat can I help you with today? \n"
 }
 role: "model",
 parts {
   text: "i m raghav"
 }
 role: "user",
 parts {
   text: "Nice to meet you, Raghav! What can I do for you today?  Do you have any questions, need help with something, or want to chat? \n"
 }
 role: "model"]

In [11]:
type(chat.history[0])

google.ai.generativelanguage_v1beta.types.content.Content

In [2]:
chat = model.start_chat(history=[])
response = chat.send_message('hi', stream=True)

for chunk in response:
  print(chunk.text)


Hi
! How can I help you today? 



In [4]:
len(chat.history)

2

In [49]:
type(chat.history[0])

google.ai.generativelanguage_v1beta.types.content.Content

In [20]:
import google
def serialize_content(content_obj):
    if isinstance(content_obj, list):
        return [serialize_content(item) for item in content_obj]
    elif isinstance(content_obj, dict):
        return {key: serialize_content(value) for key, value in content_obj.items()}
    elif isinstance(content_obj, content_module.Content):
        # Manually serialize known fields of the Content object
        return {
            'role': content_obj.role,
            'parts': [{'text': part.text} for part in content_obj.parts]
        }
    else:
        return content_obj

In [19]:
dir(chat.history[0])

['__bool__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'parts',
 'role']

In [23]:
serialize_content(chat.history)

NameError: name 'content_module' is not defined

In [26]:
response = chat.send_message('how is epsilon changed, what is the function that does that show me that', stream=True)

for chunk in response:
  print(chunk.text)

You
're right to focus on `epsilon`. It's the step size used
 in the leapfrog integration, and its value significantly affects the performance of the NUT
S sampler.  Here's how `epsilon` is changed, and the function responsible:

**1. Initial Value:**

   * The `find
_reasonable_epsilon` function is called initially to establish a starting value for `epsilon`. This function employs a heuristic approach:
      * It starts with a
 relatively large `epsilon` value.
      * It repeatedly halves `epsilon` until it finds a value where the leapfrog step doesn't lead to infinite or NaN likelihood values (ensuring numerical stability).
      * It then adjusts
 `epsilon` further, increasing or decreasing it until it finds a value that results in an acceptance probability close to 0.5.

**2. Adaptive Tuning (Dual Averaging):**

   *  The core of `epsilon
` adaptation is handled within the `nuts6` function using a technique called **dual averaging**. 
   *  **How it Works:**
     * The algorithm k

In [27]:
response = chat.send_message('I mean tell me the filename and show me the whole relevant function for that', stream=True)

for chunk in response:
  print(chunk.text)

You
 are absolutely correct!  I apologize for the oversight in my previous response. The
 `epsilon` adaptation happens directly within the `nuts6` function, not in
 a separate function.

Here's the relevant part of the code from the file `nuts/nuts.py`:

```python
def nuts6(
f, M, Madapt, theta0, delta=0.6, progress=False):
    # ... other code ...

    # Choose a
 reasonable first epsilon by a simple heuristic.
    epsilon = find_reasonable_epsilon(theta0, grad, logp, f)

    # Parameters to the dual averaging algorithm.
    gamma = 0.05

    t0 = 10
    kappa = 0.75
    mu = log(10. * epsilon)

    # Initialize dual averaging algorithm.
    epsilonbar = 1
    Hbar =
 0

    for m in progress_range(1, M + Madapt, progress=progress):
        # ... other code ...

        # Do adaptation of epsilon if we're still doing burn-in.
        eta = 1. / float(m + t0)
        H
bar = (1. - eta) * Hbar + eta * (delta - alpha / float(nalpha))
        if (m <= Madapt):
            epsilon = exp(mu

# Filter common words

In [32]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /home/rka/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/rka/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [36]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

def filter_important_words(query):
    # Define the stopwords
    stop_words = set(stopwords.words('english'))
    # Tokenize the query
    word_tokens = word_tokenize(query)
    # Filter out the stopwords
    important_words = [word for word in word_tokens if word.lower() not in stop_words and word not in string.punctuation]
    return important_words


In [37]:

# Example usage
# query = "The quick brown fox jumps over the lazy dog"
query = "How is sampling done in the repo?"

important_words = filter_important_words(query)
print(important_words)


['sampling', 'done', 'repo']


In [51]:
repo_url = "https://github.com/mfouesneau/NUTS"

# Remove trailing slash if present
repo_url = repo_url.rstrip('/')

# Extract username and reponame from repo_url
parts = repo_url.split('/')
username = parts[-2]
reponame = parts[-1]

# Combine username and reponame
combined_string = f"{username}+{reponame}"

print(f"Combined string: {combined_string}")


Combined string: mfouesneau+NUTS


In [1]:
from repo_utils import *

[nltk_data] Downloading package stopwords to /home/rka/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/rka/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
repo_dict = {}


In [35]:
repo_url = "https://github.com/joshspeagle/dynesty"  # Replace with your repository URL
clone_path = ""
if clone_path == "":
      clone_path = f"./repo/{get_clone_path_dir(repo_url)}"
      if not os.path.exists(clone_path):
         print(f"Cloing {repo_url} at {clone_path}")
         clone_github_repo(repo_url, clone_path)
         print(f"Repository cloned to {clone_path}")


if not repo_dict:
          repo_dict = create_file_content_dict(clone_path)
          print('repo_dict created')

Cloing https://github.com/joshspeagle/dynesty at ./repo/joshspeagle+dynesty
Repository cloned to ./repo/joshspeagle+dynesty
Repository cloned to ./repo/joshspeagle+dynesty


In [36]:
user_query = "what are the kinds of bounding classes used in the codebase?"
important_words = filter_important_words(user_query)
relevant_code = search_and_format(repo_dict, important_words)

In [37]:
len(relevant_code)

578853

In [38]:
key_string = "\n".join(repo_dict.keys())

get_files_prompt = f"{key_string}.\n Above is the file structure of github codebase. To answer {user_query}, what files might be required. Reply the filenames as a python array."

In [39]:
response = model.generate_content(get_files_prompt, stream=True)
for chunk in response:
  print(chunk.text)

```
python
[
    'pyvbmc/priors/uniform_box
.py',
    'pyvbmc/priors/smooth_box
.py',
    'pyvbmc/priors/trapezoidal.py',
    'pyvbmc/priors/spline_tra
pezoidal.py',
]
``` 



In [40]:
print(response.text)

```python
[
    'pyvbmc/priors/uniform_box.py',
    'pyvbmc/priors/smooth_box.py',
    'pyvbmc/priors/trapezoidal.py',
    'pyvbmc/priors/spline_trapezoidal.py',
]
``` 



In [41]:
import re

# Example text containing the Python array

# Regex pattern to match the Python array
pattern = re.compile(r'\[\s*([\s\S]*?)\s*\]', re.MULTILINE)

# Extract the array content
text = response.text
match = pattern.search(text)
if match:
    array_content = match.group(1)
    
    # Split the content by comma and strip quotes and whitespace
    array_elements = [element.strip().strip("'\"") for element in array_content.split(',') if element.strip()]
    
    print(array_elements)
else:
    print("No array found in the text.")


['pyvbmc/priors/uniform_box.py', 'pyvbmc/priors/smooth_box.py', 'pyvbmc/priors/trapezoidal.py', 'pyvbmc/priors/spline_trapezoidal.py']


In [42]:
print(repo_dict.get(array_elements[0]))

from textwrap import indent

import numpy as np

from pyvbmc.formatting import full_repr
from pyvbmc.priors import Prior, tile_inputs


class UniformBox(Prior):
    """Multivariate uniform-box prior.

    A prior distribution represented by box of uniform dimension, with lower
    bound(s) ``a`` and upper bound(s) ``b``.

    Attributes
    ----------
    D : int
        The dimension of the prior distribution.
    a : np.ndarray
        The lower bound(s), shape `(1, D)`.
    b : np.ndarray
        The upper bound(s), shape `(1, D)`.
    """

    def __init__(self, a, b, D=None):
        """Initialize a multivariate uniform-box prior.

        Parameters
        ----------
        a : np.ndarray | float
            The lower bound(s), shape `(D,)` where `D` is the dimension
            (parameters of type ``float`` will be tiled to this shape).
        b : np.ndarray | float
            The upper bound(s), shape `(D,)` where `D` is the dimension
            (parameters of type ``float

In [32]:
print(f"The first element of the array is: {array_content[0]}")
print(f"The string you are comparing it to is: 'pyvbmc/variational_posterior/variational_posterior.py'")
print(f"The difference is: {array_content[0] != 'pyvbmc/variational_posterior/variational_posterior.py'}")

The first element of the array is: '
The string you are comparing it to is: 'pyvbmc/variational_posterior/variational_posterior.py'
The difference is: True


In [30]:
for k in repo_dict.keys():
    print(k)

.gitignore
LICENSE
.coveragerc-html.css
.pre-commit-config.yaml
setup.py
logo.svg
MANIFEST.in
CITATION.cff
pylintrc
.gitattributes
README.md
pyproject.toml
.coveragerc
docsrc/Makefile
docsrc/make.bat
docsrc/source/documentation.rst
docsrc/source/development.rst
docsrc/source/index.rst
docsrc/source/about_us.rst
docsrc/source/conf.py
docsrc/source/examples.rst
docsrc/source/quickstart.rst
docsrc/source/installation.rst
docsrc/source/css/custom.css
docsrc/source/api/advanced_docs.rst
docsrc/source/api/options/vbmc_options.rst
docsrc/source/api/functions/decorators.rst
docsrc/source/api/functions/active_sample.rst
docsrc/source/api/functions/get_hpd.rst
docsrc/source/api/functions/whitening.rst
docsrc/source/api/functions/kde_1d.rst
docsrc/source/api/functions/create_vbmc_animation.rst
docsrc/source/api/functions/entropy.rst
docsrc/source/api/functions/functions.rst
docsrc/source/api/classes/variational_posterior.rst
docsrc/source/api/classes/options.rst
docsrc/source/api/classes/iteratio

In [2]:
repo_url = 'https://github.com/rakaar/fly-courtship-quantify/tree/make_stationary_vids'
repo_url = repo_url.rstrip('/')


parts = repo_url.split('/')
username = parts[3]
reponame = parts[4]

# Check if the URL contains a branch
if len(parts) > 5 and parts[5] == 'tree':
    branchname = parts[6]
    combined_string = f"{username}+{reponame}+{branchname}"
else:
    combined_string = f"{username}+{reponame}"


print(combined_string)

rakaar+fly-courtship-quantify+make_stationary_vids


In [5]:
import os
os.path.exists(os.path.join('./repo', '.gitignore'))

True

In [9]:
import os
import json
import chardet
import concurrent.futures

def process_file(file_path, clone_path):
    relative_path = os.path.relpath(file_path, clone_path)
    try:
        with open(file_path, 'rb') as f:
            raw_data = f.read()
            if file_path.endswith('.ipynb'):
                # Handle Jupyter notebook files
                try:
                    content = json.loads(raw_data)
                    cell_sources = [
                        ''.join(cell.get('source', ''))
                        for cell in content.get('cells', [])
                        if cell.get('cell_type') in ('markdown', 'code')
                    ]
                    text = '\n'.join(cell_sources)
                    return relative_path, text
                except json.JSONDecodeError as e:
                    print(f"Failed to parse notebook {file_path}: {e}")
                    return None
            else:
                # Handle other text files
                result = chardet.detect(raw_data)
                encoding = result['encoding']
                if encoding is not None:
                    text = raw_data.decode(encoding)
                    return relative_path, text
                else:
                    print(f"Skipping non-text file: {file_path}")
                    return None
    except Exception as e:
        print(f"Failed to read {file_path}: {e}")
        return None

def create_file_content_dict(clone_path):
    file_content_dict = {}
    files_to_process = []

    for root, _, files in os.walk(clone_path):
        if '.git' in root:
            continue
        for file in files:
            file_path = os.path.join(root, file)
            print('file_path: ',file_path)
            files_to_process.append(file_path)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {executor.submit(process_file, file_path, clone_path): file_path for file_path in files_to_process}
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            if result is not None:
                relative_path, text = result
                file_content_dict[relative_path] = text

    return file_content_dict


In [2]:
from repo_utils import clone_github_repo
clone_github_repo('https://github.com/rakaar/rakaar.github.io','../repo/temp')

[nltk_data] Downloading package stopwords to /home/rka/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/rka/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Repository cloned to ../repo/temp


In [11]:
dd = create_file_content_dict('../repo/temp')

inside 1st for loop
file_path:  ../repo/temp/.gitignore
file_path:  ../repo/temp/.editorconfig
file_path:  ../repo/temp/love.md
file_path:  ../repo/temp/minima.gemspec
file_path:  ../repo/temp/readings.md
file_path:  ../repo/temp/Gemfile
file_path:  ../repo/temp/evernotes.md
file_path:  ../repo/temp/about.md
file_path:  ../repo/temp/use_rss.md
file_path:  ../repo/temp/LICENSE.txt
file_path:  ../repo/temp/404.html
file_path:  ../repo/temp/index.md
file_path:  ../repo/temp/CODE_OF_CONDUCT.md
file_path:  ../repo/temp/screenshot.png
file_path:  ../repo/temp/.travis.yml
file_path:  ../repo/temp/README.md
file_path:  ../repo/temp/_config.yml
inside 1st for loop
file_path:  ../repo/temp/_posts/2020-03-30-eudyptula-challenges-task-01.md
file_path:  ../repo/temp/_posts/2021-04-18-on-what-2-laugh.md
file_path:  ../repo/temp/_posts/2022-02-01-genes-say-beautiful.md
file_path:  ../repo/temp/_posts/2020-12-02-addingmathjax.md
file_path:  ../repo/temp/_posts/2021-02-12-measure-m-outof-n.md
file_path

In [12]:
dd

{'use_rss.md': '<!-- ---\nlayout: page\ntitle: RSS\npermalink: /rss/\n---\n\nLink to my RSS feed - [https://rakaar.github.io/feed.xml](https://rakaar.github.io/feed.xml)\n\nI have written [here](https://rakaar.github.io/posts/2021-07-23-use-rss/) in detail about why you should use RSS and why it is effective. If you not familiar with RSS watch [this](https://www.youtube.com/watch?v=6HNUqDL-pI8), do encourage your friends to use RSS. You can share your RSS feed with me via email - rka87338@gmail.com\n\nBoth the web standards - RSS and Atom are equally good. You can use any one of them. Technically, though both of them are different web standards, RSS is a more common name. RSS is synonymous with the idea of content syndication on the web.\n -->\n',
 '_posts/2021-02-12-measure-m-outof-n.md': "---\nlayout: post\ntitle:  Measuring only m qubits out of n regiters in Qiskit\nsubtitle: Also a bit about order reversal in measurement\ndate: 2021-02-12\ncategories: [quantum]\n---\n\n\nSuppose yo

In [13]:
print(dd['readings.md'])

---
layout: page
title: Readings
permalink: /readings/
---



## Dune Messiah by Frank Herbert

This book was hard to read. At many places, it is wordy and vague. Online forums  were [Jagurutu](http://www.jacurutu.com/index.php?sid=6960afbf721ee245f1fc5b27530f224e) were a great aid. I am still frustrated by the fact that fantasy elements like *prescience* have played a major role in key parts of a **"sci-fi"** book. Yet, I will continue with the next part because I have more questions that   haven't been answered. 

## Why we die by Venki Ramakrishnan

I picked this book because I loved Venki's first book- The Gene Machine. This book repeatedly proves the point that a good writing on an unfamiliar topic is much more enjoyable bad writing on a exciting topic.

The book can be considered as a crash course in cell biology. The author explains different causes of aging and death at the level of activities that go wrong within a cell - accumulation of waste, too many misfolded proteins, DNA

In [15]:
from repo_utils import does_file_exist

In [19]:
pkl_filename = f"{'rakaar+fly-courtship-quantify'}.pkl"
for filename in os.listdir('./repo'):
    print(filename)

.git
rakaar+fly-courtship-quantify+make_stationary_vids.pkl
rakaar+fly-courtship-quantify.pkl


In [22]:
os.path.exists(os.path.join('./repo', pkl_filename))

True

In [23]:
does_file_exist('./repo', pkl_filename)

True

In [21]:
pkl_filename == 'rakaar+fly-courtship-quantify.pkl'

True